In [1]:
FREECADPATH = 'C:/Users/Slashito/Documents/FreeCAD_0.19.24219_Win-LPv12.4.3/bin'

import sys
sys.path.append(FREECADPATH)

# Macro Begin: C:\Users\Slashito\AppData\Roaming\FreeCAD\Macro
import FreeCAD as App
import FreeCAD, Sketcher

import FabCAD #Custom Library

FabCAD.nuevoDocumento().nuevaPieza()

nameProject = "FirstSteps"
namePart = "Pieza01"
nameBody = "Cuerpo"

# Crear un nuevo documento en blanco
doc = FabCAD.nuevoDocumento(nameProject)
doc.nuevaPieza(namePart).nuevoCuerpo(nameBody).crearPlano("Planta").crearPlano("Alzado", soporte="XZ_Plane", cuerpo = nameBody).crearPlano("Vista_Lateral", "YZ_Plane", doc.cuerpos[nameBody]).nuevoSketch("sketch", doc.cuerpos[nameBody], doc.planos["Planta"])
sketch = doc.seleccionarObjeto("sketch")

puntos = []
puntos.append([29.739,8.8786])
puntos.append([29.739,71.7241])
puntos.append([117.794,71.7241])
puntos.append([136.967,52.5509, 19.17, False])
puntos.append([136.967,8.8786])
puntos.append([29.739,8.8786])

sketch.crearPolilinea(puntos).extrusionAditiva("Test", 50, planoMedio = 1)

sketch02 = doc.nuevoSketch("sketch02", soporte = "Planta").seleccionarObjeto("sketch02").crearPoligono(10,[50,50], 7).extrusionDeVaciado("Test02", 50, planoMedio = 1)


In [2]:
for llave, item in doc.planos.items(): print(f"{llave}")

Planta
Alzado
Vista_Lateral


In [4]:
nuevoSketch = doc.nuevoSketch("sketch", doc.cuerpos[nameBody], doc.planos["Planta"])

In [5]:
nuevoSketch.crearCirculo([150, 50], 15)

print(nuevoSketch.contGeometria)

puntos = []

puntos.append([47,40])
puntos.append([54,55])
puntos.append([64,33])
puntos.append([69,48])
puntos.append([74,38])
puntos.append([83,60])

nuevoSketch.crearPolilinea(puntos)

print(nuevoSketch.contGeometria)

1
6


In [6]:
sumaX = 0
longitudH = 20
longitudLinea = 10

#Note the use of FreeCAD original library inside the custom library wrap inside doc.base variable 
for linea in range(3):
	nuevoSketch.crearLinea( [ (2 + sumaX) , 2] , [(12 + sumaX), 12] )
	doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Distance', linea, longitudLinea)) 

	if linea > 0:
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Parallel',  linea -1, linea)) 
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Horizontal', linea - 1, 1, linea, 1)) 		
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('DistanceX', linea - 1, 1, linea, 1, App.Units.Quantity('{} mm'.format( longitudH )))) 

	sumaX =sumaX + 20
	longitudLinea += 10

doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Angle',0,1,-1,2, App.Units.Quantity('135.000000 deg'))) 


13

# Crear ranuras
## Para crear ranuras el metodo interno de la libreria FabCAD hace lo siguiente:

> Primero que se han recibido los dos puntos de inicio y fin de la ranura se calcula su pendiente, esto con la finalidad de obtener los angulos de inicio y fin de los arcos, una vez que se obtiene esta pendiente se le agregan 90 grados para que los arcos sean perpendiculares a la linea recta entre los dos puntos y se colocan los dos puntos de inicio y fin de cada arco, por ultimo segun el radio indicado se coloca otro punto colineal a la linea recta entre los dos puntos y con la longitud del radio especificado, luego estos puntos se bloquean para que queden en la posicion indicada y se dibuja toda la geometria a la que posteriormente se le dará restricciones de posicion

## Ejemplo:
### Se quiere dibujar una ranura entre los puntos [2,1] y [8,5] con un radio de 2mm

> Lo primero que hace el metodo es calcular m y luego los angulos:

`m = (puntoFinal[1] - puntoInicial[1]) / (puntoFinal[0] - puntoInicial[0])`
`angulos = [m+(math.pi/2), m+(math.pi/2)+math.pi]`

> Con esto listo solo se dibujan los dos arcos y las dos lineas, posteriormente se les agrega las restricciones

`conList = []`

        conList.append(Sketcher.Constraint('Equal',contGeometria,contGeometria+1))
        conList.append(Sketcher.Constraint('Radius',contGeometria, radio))
        conList.append(Sketcher.Constraint('Tangent',contGeometria,1,contGeometria+2,1))
        conList.append(Sketcher.Constraint('Tangent',contGeometria,2,contGeometria+3,1))
        conList.append(Sketcher.Constraint('Tangent',contGeometria+1,1,contGeometria+3,2))
        conList.append(Sketcher.Constraint('Tangent',contGeometria+1,2,contGeometria+2,2))



In [17]:
nuevoSketch.crearRanura([2,1], [8,5], 2)

PuntoUno: 2.237462993461563, PuntoDos: 5.379055647051356


# Matriz Lineal
## Para utilizar la funcion de matriz lineal se debe seguir la siguiente estructura:

`s = doc.nuevoSketch("sketch", cuerpo, plano)`

`s.matrizLineal(elementos, colxfil, distancia, direccion = 0, pivote = None, clonar = False, acotar = False)`

> Como se puede observar es una funcion que puede recibir muchos parametros, sin embargo, no siempre se tienen que usar todos, como se puede observar tambien muchos de los parametros ya estan rellenos con un valor predeterminado con los valores que mas usualmente se usan, por ejemplo la direccion (0 = Horizontal), el punto pivote (None quiere decir que se utiliza el primer punto del primer elemento) y lo mismo con clonar y acotar, por lo que los argumentos minimos serian:

`s.matrizLineal(elementos, colxfil, distancia)`


## A continuacion se muestran ejemplos de uso con la geometria de un poligono:
`s.crearPoligono(10, [00,200] 5)`
![Geometria Base](./img/ejemploMatrizLineal01.png "Geometria Base")

1. ### Creacion de matriz Horizontal con relaciones de posicion respecto a la geometria original y acotaciones
`s.matrizLineal([0,1,2,3,4,5], 5, 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal02.png "Resultado Ejemplo 01")

2. ### Creacion de matriz con 25 grados de inclinacion
`s.matrizLineal([0,1,2,3,4,5], 5, 27.5, 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal03.png "Resultado Ejemplo 02")

3. ### Creacion de matriz de 3 filas y 3 columnas con 25 grados de inclinación y diferentes distancias entre filas y columnas
`s.matrizLineal([0,1,2,3,4,5], [3,3], [27.5, 40], 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal04.png "Resultado Ejemplo 03")

In [1]:
puntos = []
puntos.append([29.739,8.8786])
puntos.append([29.739,71.7241])
puntos.append([117.794,71.7241])
puntos.append([136.967,52.5509, 19.17, False])
puntos.append([136.967,8.8786])
puntos.append([29.739,8.8786])

nuevoSketch.crearPolilinea(puntos)

puntos = []
puntos.append([49.0133,7.76455])
puntos.append([88.7014,76.3167])
puntos.append([126.946,-15.6875])
puntos.append([150.398,46.731])
puntos.append([159.418,33.7422, 9.61, True])
puntos.append([207.405,18.9494])
puntos.append([222.919,66.5751])
puntos.append([238.073,61.8847, 7.93, True])

nuevoSketch.crearPolilinea(puntos)

puntos = []
puntos.append([47,40])
puntos.append([54,55])
puntos.append([64,33])
puntos.append([69,48,10, True])
puntos.append([74,38])
puntos.append([83,60])
puntos.append([1,2,3,4])

nuevoSketch.crearPolilinea(puntos)

#Puntos de Prueba 07/04/2021
puntos = []
puntos.append([50.8234,33.2904])
puntos.append([105.665,103.646])
puntos.append([142.828,-3.5113])
puntos.append([177.104,74.7824])
puntos.append([191.175,50.248, 20, True])
puntos.append([244.213,9.83832])
puntos.append([244.213,87.4105])

nuevoSketch.crearPolilinea(puntos)

In [140]:
#Obtener el centro de la circunferencia dados 3 puntos
#Util para construir el arco 3 puntos
import math

#Funcion para obtener pendiente, radianes y grados de una recta
def datos(v1,v2):
    try:
        m = (v2[1]-v1[1])/(v2[0]-v1[0])
        rad = math.atan(m)
        deg = math.degrees(rad)
    except:
        m = "Indeterminada"
        rad = 0
        deg = 0

    return [m, rad, deg]

p1 = [10,10]
p2 = [12.5,7.5]
p3 = [11, 14]

test01 = [199.66,65.5034]
test02 = [220.712,57.2334]

mTest, radTest, degTest = datos(test01, test02)
radTestRoted = radTest + math.pi/2

xTest = centroTest[0] + math.cos(radTestRoted)
yTest = centroTest[1] + math.sin(radTestRoted)

print(f"Angulo de la pendiente en radianes: {radTest}")
print(f"Posicion del punto final de la linea tangente: {[xTest, yTest]}")

#Estos puntos se extrajeron manualmente pero en la libreria con la funcion de crearLineaCentro se puede extraer automaticante
centroP1P2 = [((p1[0] + p2[0])/2), ((p1[1] + p2[1])/2)]
centroP2P3 = [((p2[0] + p3[0])/2), ((p2[1] + p3[1])/2)]

#Se obtienen los datos necesarios mediante la funcion implementada en el inicio del metodo
m12, rad12, deg12 = datos(p1,p2)
m23, rad23, deg23 = datos(p2,p3)

#Datos para la formula de intersección de dos rectas
A = math.tan(rad12-(math.pi/2))
B = math.tan(rad23-(math.pi/2))
C = (-A * centroP1P2[0])  + centroP1P2[1]
D = (-B * centroP2P3[0])  + centroP2P3[1]

posCentro = [(D - C) / (A - B)]
posCentro.append(A*posCentro[0]+(-A*centroP1P2[0] +centroP1P2[1]))

#Obtención de los angulos para el arco
angulos = [datos(posCentro, p3)[1]]
angulos.append( math.pi + datos(posCentro, p1)[1] )

radio = math.dist(posCentro, p3)

print()
print(radio)
print(posCentro)
print(angulos)

Angulo de la pendiente en radianes: -0.37431597999720395
Posicion del punto final de la linea tangente: [90.17633595762312, 25.30890794194464]

3.8897300677553437
[13.699999999999998, 11.2]
[-0.8035779784704272, 3.4552133058064913]


In [ ]:
#X_CENTRO   <   X_FINAL
    #     Y_CENTRO  <   Y_FINAL
    puntos = []
    puntos.append([177.104,74.7824])
    puntos.append([191.175,50.248])
    puntos.append([187.206,64.2741])

    nuevoSketch.crearArcoTresPuntos( [177.104,74.7824],[191.175,50.248],[187.206,64.2741])

    #     Y_CENTRO  >   Y_FINAL
    puntos = []
    puntos.append([200.292,88.1126])
    puntos.append([220.712,57.2334])
    puntos.append([221.241,72.2737])

    nuevoSketch.crearArcoTresPuntos( [200.292,88.1126],[220.712,57.2334],[221.241,72.2737])

#X_CENTRO   >   X_FINAL
    #     Y_CENTRO  <   Y_FINAL
    puntos = []
    puntos.append([32.5972,11.8399])
    puntos.append([31.5169,8.33118])
    puntos.append([37.2923,8.53185])

    nuevoSketch.crearArcoTresPuntos( [32.5972,11.8399],[31.5169,8.33118],[37.2923,8.53185])

    #     Y_CENTRO  >   Y_FINAL
    puntos = []
    puntos.append([10,10])
    puntos.append([12.5,7.5])
    puntos.append([11,14])

    nuevoSketch.crearArcoTresPuntos( [10,10],[12.5,7.5],[11,14])

# INTRODUCCION AL MODELADO 3D
## PRIMEROS METODOS DE EXTRUSION
1. ### ExtrusionAditiva 
2. ### ExtrusionDeVaciado 
3. ### RevolucionAditiva 

En esta actualizacion se han corregido muchos errores de sintaxis en la forma de manejar los erroes y de nombres de variables repetidos en ciclos anidados, **pero lo mas importante es la inclusión de nuevos metodos que ayudan a pasar al mundo 3D los croquis que hemos hecho hasta ahora**

Estos metodos pueden ser usados de diferentes formas pero la mas habitual con seguridad será llamarlo desde el objeto de sketch en el que hayamos creado la geometria:

#### Ejemplos de cada metodo:
Teniendo un sketch para cada ejemplo del tipo:

`s = doc.nuevoSketch("sketch", cuerpo, plano)`

1. ### ExtrusionAditiva

    `puntos = []`

    `puntos.append([29.739,8.8786])`

    `puntos.append([29.739,71.7241])`

    `puntos.append([117.794,71.7241])`

    `puntos.append([136.967,52.5509, 19.17, False])`

    `puntos.append([136.967,8.8786])`

    `puntos.append([29.739,8.8786])`

    `s.crearPolilinea(puntos)`

    `s.extrusionAditiva("Test", 50, planoMedio = 1)`

2. ### ExtrusionDeVaciado

    `s.crearPoligono(10,[50,50], 7)`

    `s.extrusionDeVaciado("Test02", 50, planoMedio = 1)`

3. ### RevolucionAditiva

    `s.crearLinea([152.934,38.4578], [102.066,1.54221])`

    `s.crearArcoTresPuntos([152.934,38.4578], [102.066,1.54221], [148.132202,-3.356926])`

    `s.revolucionAditiva()`

### Todos estos ejemplos juntos darian como resultado la siguiente forma:

![EjemploExtrusiones](./img/ejemploExtrusiones01.png "Ejemplo de Extrusiones")

